# Using TempestExtremes' NodeFileCompose to extract cyclone-centered snapshots

In this example, we extract SLP snapshots along the tracks in `TC_tracks/um_glm_n1280_GAL9_sample.csv`, which is a subset of three tracks.

Snapshots should be retrieved for all TC points listed in the tracks csv, over a radius of 10°GCD around the cyclone center. 
Snapshots must be obtained from data at a resolution close to the simulation resolution in the tropics, 
and choosing NodeFileCompose parameters so that the resolution of the snapshots is similar to that of the simulation.

In this case, we have pre-processed the data over a 0.1x0.1° grid. 

In [1]:
# Script Parameters
run='um_glm_n2560_RAL3p3' # Code of your simulation (must be the reference in the catalog)
var="pr"

# Folders
scr_dir='/work/scratch-nopw2/sbourdin/' # Change to your own scratch/temporary folder
data_pp_dir=$scr_dir/$run/data_pp # dir to store preprocessed (pp) files

# Input
tracks_file="../TC_tracks/$run.csv"
#tracks_file="../TC_tracks/${run}_sample.csv"

# Output
if ! [ -d ../snapshots/${var}/ ]; then mkdir ../snapshots/${var}/; fi
snaps_file="../snapshots/${var}/${run}_${var}.nc"

Before you start, make sure you have preprocessed the data you need with `TE_pre-processing.ipynb`: You must have the fields covering all the time steps in your tracks.

In [2]:
# List pre-processed files for this variable
ls $data_pp_dir/*$var*

/work/scratch-nopw2/sbourdin//um_glm_n2560_RAL3p3/data_pp/um_glm_n2560_RAL3p3_pr_2020-01-20T00.nc
/work/scratch-nopw2/sbourdin//um_glm_n2560_RAL3p3/data_pp/um_glm_n2560_RAL3p3_pr_2020-01-20T06.nc
/work/scratch-nopw2/sbourdin//um_glm_n2560_RAL3p3/data_pp/um_glm_n2560_RAL3p3_pr_2020-01-20T12.nc
/work/scratch-nopw2/sbourdin//um_glm_n2560_RAL3p3/data_pp/um_glm_n2560_RAL3p3_pr_2020-01-20T18.nc
/work/scratch-nopw2/sbourdin//um_glm_n2560_RAL3p3/data_pp/um_glm_n2560_RAL3p3_pr_2020-01-21T00.nc
/work/scratch-nopw2/sbourdin//um_glm_n2560_RAL3p3/data_pp/um_glm_n2560_RAL3p3_pr_2020-01-21T06.nc
/work/scratch-nopw2/sbourdin//um_glm_n2560_RAL3p3/data_pp/um_glm_n2560_RAL3p3_pr_2020-01-21T12.nc
/work/scratch-nopw2/sbourdin//um_glm_n2560_RAL3p3/data_pp/um_glm_n2560_RAL3p3_pr_2020-01-21T18.nc
/work/scratch-nopw2/sbourdin//um_glm_n2560_RAL3p3/data_pp/um_glm_n2560_RAL3p3_pr_2020-01-22T00.nc
/work/scratch-nopw2/sbourdin//um_glm_n2560_RAL3p3/data_pp/um_glm_n2560_RAL3p3_pr_2020-01-22T06.nc
/work/scratch-nopw2/

In [3]:
## Prepare file list
flist=`ls $data_pp_dir/*$var*`
INPUT=""
for f in $flist
do
INPUT="$INPUT$f;"
done
INPUT=${INPUT:0:-1} # Remove last ;
echo $INPUT > INPUT.txt

In NodeFileCompose, we'll use an XY (square) box centered on the cyclone center. 
To avoid loss of information as much as possible, `dx` should be set to be close to the model resolution. Note `dx` is in °GCD. 
Then, adjust `resx` to make sure 1000km=10°GCD.
For example, for a 10km grid, `dx=0.1` and `resx=100`.

In [5]:
conda run -n hackathon NodeFileCompose \
--in_nodefile "$tracks_file" \
--in_nodefile_type "SN" \
--in_fmt "(auto)" \
--in_data_list "INPUT.txt" \
--out_data "$snaps_file" \
--var "$var" \
--varout "$var" \
--out_grid "XY" \
--dx 0.05 \
--resx 200 \
--snapshots


CondaError: KeyboardInterrupt

